# objectif du notebook
- création les jeux de données à annoter manuellement sur excel
    - distribution aléatoire entre N annotateurs

# création les jeux de données à annoter manuellement sur excel

In [1]:
import os 
import random
import itertools
import uuid
import pandas as pd

## chargement des données et prétraitement

In [4]:
ecole_des_loisirs = ['âne',
 'animaux des champs et des bois',
 'animaux divers',
 'animaux domestiques',
 'araignée',
 'autruche, émeu',
 'baleine',
 'belette',
 'bison',
 'blaireau',
 'caméléon',
 'canard, oie',
 'castor',
 'cervidés',
 'chameau, dromadaire',
 'chamois',
 'chat',
 'chauve souris',
 'cheval',
 'chien',
 'cochon',
 'coyote, chacal',
 'crabe',
 'crocodile, alligator',
 'dauphin',
 'des animaux et des hommes',
 'dindon',
 'dinosaure (animal)',
 'écureuil',
 'éléphant',
 'escargot',
 'félins divers',
 'girafe',
 'gnou',
 'grenouille, crapaud, têtard',
 'hamster, cobaye',
 'hérisson, porc-épic',
 'hibou',
 'hippocampe',
 'hippopotame',
 'insectes divers',
 'kangourou',
 'koala',
 'lamantin',
 'lapin, lièvre',
 'lézard',
 'lion',
 'loir, lérot',
 'loup',
 'loutre',
 'lynx',
 'mammouth',
 'marmotte',
 'morse',
 'mouton, chèvre, bouc',
 'oiseaux divers',
 'opossum',
 'ornithorynque',
 'orque',
 'ours',
 'panda',
 'papillon',
 'paresseux (animal)',
 'phoque',
 'pieuvre, poulpe',
 'pingouin, manchot',
 'poissons divers',
 'poule, poussin, coq',
 'raton laveur',
 'renard',
 'requin',
 'rhinocéros',
 'sanglier',
 'serpent',
 'singe, chimpanzé, gorille',
 'souris, mulot, rat, campagnol',
 'suricate, mangouste',
 'tatou',
 'taupe',
 'tigre',
 'tortue',
 'vache, veau, taureau',
 'ver de terre, lombric',
 'yéti',
 'zèbre',
 'art pariétal, peinture rupestre',
 'art traditionnel, arts premiers',
 'cinéma',
 'cirque',
 'clown',
 "commedia dell'arte",
 'créativité, création artistique',
 'danse',
 'funambule',
 'guignol',
 'Littérature, livre, écrivain',
 'marionnette',
 'mime',
 'musique',
 'opéra',
 'peinture',
 'photographie',
 'photographie (livre de)',
 'sculpture',
 'street-art',
 'théâtre (activité)',
 'dent, dent de lait',
 'main',
 'schéma corporel',
 'squelette',
 'hygiène, propreté',
 'pipi, caca, toilettes',
 'pot',
 'poux',
 'beauté, laideur',
 'coquetterie',
 'imagerie du corps',
 'autisme',
 'corps et ses différences (le)',
 'handicap',
 'lunettes',
 'grossesse, avortement',
 'naissance, maternité',
 'accident',
 'alcoolisme',
 'anorexie',
 'appétit',
 'bobo',
 'boulimie',
 'drogue, toxicomanie',
 'épidémie',
 'hospitalisation',
 'maladie, docteur',
 'pipi au lit, énurésie',
 'santé',
 'sida',
 'homosexualité',
 'puberté, transformation du corps',
 'sexualité',
 'dormir, coucher',
 'peur du noir, cauchemar',
 'rêve',
 'les cinq sens',
 'prévention (le corps)',
 'repère spatio temporel',
 'taille',
 "apprentissage de l'autonomie",
 'apprentissage de la vie',
 'premiers apprentissages',
 'bébé',
 'crèche',
 'tétine, biberon',
 'apprentissage des langues',
 'classe de nature',
 'collège, lycée, professeur',
 'école buissonnière',
 'école, instituteur',
 'pensionnat',
 'rentrée des classes',
 'sortie scolaire',
 'université',
 'mort, deuil',
 'prise de responsabilité',
 'temps (qui passe)',
 'vieillesse',
 'généalogie',
 'recherche de ses origines',
 'bêtises, désobéissance',
 'conflit parents, enfants',
 'fratrie',
 'fugue',
 'grand-mère',
 'grand-père',
 'grands-parents',
 'homoparentalité',
 'jumeaux, jumelles',
 'naissance, nouveau bébé dans la fratrie',
 'relation avec la mère',
 'relation avec le père',
 'relation enfant, adulte',
 'sœurs et frères : aîné, cadet',
 'déménagement',
 'désordre',
 'vie quotidienne',
 'adoption',
 'dispute',
 'divorce, séparation',
 'en famille',
 'enfant abandonné, abandon',
 'enfant maltraité',
 "famille d'accueil",
 'famille éclatée',
 'famille monoparentale',
 'famille recomposée, remariage',
 'la famille',
 'mariage',
 'orphelin, orpheline',
 'secrets de famille',
 'vacances en famille',
 'aventure et mystère',
 "cape et d'épée",
 'histoire policière',
 'récit de voyage',
 "roman d'aventure",
 'conte détourné',
 'conte détourné, le bal des 12 princesses',
 'conte détourné, barbe bleue',
 'conte détourné, blanche-neige',
 "conte détourné, boucle d'or et les 3 ours",
 'conte détourné, cendrillon',
 'conte détourné, la belle au bois dormant',
 'conte détourné, la princesse au petit pois',
 'conte détourné, le loup et les 7 cabris',
 'conte détourné, le petit chaperon rouge',
 'conte détourné, le petit poucet',
 'conte détourné, les 3 petits cochons',
 'conte détourné, les fées',
 'contes du monde entier',
 'contes et recueil de contes',
 'fables',
 'classique abrégé',
 'classique intégral',
 "classiques du livre d'images",
 'fantastique',
 'fantasy',
 'science-fiction',
 'fables, fabliaux',
 'Littérature du Moyen Âge',
 'roman de la table ronde',
 'cow-boy',
 'super-héros',
 'ange',
 'chevalier',
 'diable',
 'dragon',
 'fantôme',
 'fée',
 'géant',
 'génie',
 'licorne',
 'lutin, troll, elfe, gnome',
 'magicien, magie',
 'monstre',
 'nain',
 'ogre',
 'personnage de conte (divers)',
 'prince, princesse',
 'roi, reine',
 'sirène',
 'sorcier, sorcière',
 'vampire',
 'sketch',
 'théâtre classique',
 'théâtre contemporain',
 'théâtre, pièce de théâtre',
 'atlas',
 'autobiographie',
 'aventure',
 'bande dessinée',
 'biographie',
 'comptine',
 'conte',
 'énigmes',
 'épouvante',
 'extraterrestre',
 'humour',
 'journal',
 'lettres, correspondance',
 'livre dont on est le héros',
 'Livre ukrainien',
 'Livre-jeu',
 'MOOC',
 'mythologie',
 'parodie, pastiche',
 'philosophie',
 'poésie',
 'proverbe',
 "recueil d'histoires",
 'recueil de nouvelles',
 'rime',
 "roman de cap et d'épée",
 'texte fondateur',
 'Afrique (continent africain)',
 'Afrique du nord',
 'Afrique du sud',
 'Algérie',
 'Egypte',
 'Kenya',
 'Libye',
 'Madagascar',
 'Maghreb',
 'Mali',
 'Maroc',
 'Seychelles (îles des)',
 'Soudan',
 'Tanzanie',
 'Tunisie',
 'Zanzibar',
 'Antilles, Caraïbes',
 'Costa-Rica',
 'Cuba',
 'Guatemala',
 'Haïti',
 'Mexique',
 'Porto-Rico',
 'Alaska',
 'Canada',
 'Etats-Unis',
 'Argentine',
 'Bolivie',
 'Brésil',
 'Chili',
 'Colombie',
 'Equateur',
 'Pérou',
 'Vénézuela',
 'Afghanistan',
 'Arabie',
 'Arménie',
 'Asie (en général)',
 'Bahrein',
 'Bangladesh',
 'Birmanie',
 'Bornéo',
 'Cambodge',
 'Chine',
 'Corée',
 'Emirats',
 'Extrème-Orient',
 'Inde',
 'Indonésie',
 'Irak',
 'Iran',
 'Israël, palestine',
 'Japon',
 'Jordanie',
 'Kirghizistan',
 'Koweït',
 'Laos',
 'Liban',
 'Malaisie',
 'Mongolie',
 'Moyen-Orient',
 'Népal',
 'Oman',
 'Pakistan',
 'Philippines',
 'Sri lanka',
 'Syrie',
 'Taïwan',
 'Thaïlande',
 'Tibet',
 'Viêtnam',
 'Yémen',
 'Géorgie',
 'Ouzbékistan',
 'Russie',
 'Turquie',
 'URSS',
 'Esquimau, Inuit',
 'Gaulois',
 'Indien',
 "Indiens d'Amérique",
 'Mayas',
 'Monde arabe',
 'Touareg',
 'Vikings',
 'Allemagne',
 'Autriche',
 'Balkans',
 'Belgique',
 'Danemark',
 'Espagne',
 'Europe',
 'Finlande',
 'France',
 'Grande-Bretagne',
 'Grèce',
 'Hongrie',
 'Islande',
 'Italie',
 'Norvège',
 'Pays nordiques',
 'Pays-Bas',
 'Pologne',
 'Portugal',
 'Roumanie',
 'Suède',
 'Suisse',
 'Tchécoslovaquie',
 'Ukraine',
 'Amazonie',
 'banquise',
 'désert',
 'forêt',
 'île',
 'jungle, forêt lointaine',
 'montagne',
 'rivière, fleuve, étang',
 'Sahara',
 'volcan',
 'Australie',
 'Nouvelle-Calédonie',
 'Nouvelle-Guinée',
 'Nouvelle-Zélande',
 'Polynésie',
 'Arctique, Antarctique',
 'camp de concentration',
 'Commune de paris',
 'conquête',
 'déportation',
 'entre deux guerres',
 'fascisme',
 'génocide arménien',
 'génocide khmère',
 'guerre, conflit',
 'Hiroshima',
 'mafia',
 'nazisme',
 'Première Guerre mondiale',
 'révolution',
 'Révolution française',
 'Seconde Guerre mondiale',
 'terrorisme, guerre civile',
 'Antiquité',
 "Antiquité : l'Egypte",
 'Antiquité : la Gaule',
 'Antiquité : la Grèce',
 'Antiquité : Rome',
 'Gaule (la)',
 "Histoire (l'an 1000)",
 'Histoire : 13e siècle',
 'Histoire : 15e siècle',
 'Histoire : 16e siècle',
 'Histoire : 17e siècle',
 'Histoire : 18e siècle',
 'Histoire : 19e siècle',
 'Histoire : 20e siècle',
 'Histoire : 21e siècle',
 'Histoire : 5e siècle',
 'Histoire : Moyen Âge',
 'Préhistoire',
 'Martin Luther King',
 'château, château fort',
 'Cro-Magnon (Homme de)',
 'dinosaure',
 'Histoire du monde',
 'Mésopotamie',
 'personnage historique',
 'pionniers',
 'bruits et cris',
 'chant, chanson',
 'contraire',
 'couleurs',
 'étymologie',
 'histoire sans texte',
 'images pour jouer',
 'imagier',
 'langage, jeux de mots',
 'livre à compter, chiffres',
 'métamorphose (image)',
 'balle, ballon',
 'cache-cache',
 'cerf-volant',
 'course',
 'déguisement',
 'doudou',
 'jeu',
 'jouets',
 'masque',
 'ours en peluche',
 'poupée',
 'robot',
 'abécédaire',
 'almanach',
 'coloriage, livre à colorier',
 'jeu de rôle',
 'jeu vidéo',
 'les échecs',
 'livre animé',
 'livre de bain',
 'livre de naissance',
 'livre puzzle',
 'chef, meneur, leader',
 'courageux',
 'cupide',
 'curiosité',
 'égoïste',
 'fantaisie, fantaisiste',
 'farceur, farceuse',
 'ingénieux, débrouillard',
 'paresseux',
 'rabat-joie, grincheux',
 'responsable',
 'rêveur, poète',
 'rusé, astucieux',
 'vantard',
 'ami imaginaire',
 'amitié',
 'amour',
 'caprices, entêtement',
 'colère',
 'éco-anxiété',
 'émotion',
 'ennui',
 'gratitude',
 'harcèlement',
 'honte, gêne',
 'humeur',
 'jalousie',
 'méchanceté, cruauté',
 'mensonge',
 'moquerie',
 'nostalgie',
 'peur',
 'regard des autres',
 'rivalité, bagarre, dispute',
 "sentiment d'injustice",
 'sentiment de culpabilité',
 'tendresse, baiser, câlin',
 'timidité',
 'tristesse, chagrin',
 'vengeance',
 'affirmation de soi',
 'genre',
 'imagination, fabulation',
 'initiation, épreuve',
 "le complexe d'Oedipe",
 'portrait',
 "recherche d'absolu",
 "recherche d'identité",
 'secret',
 'souvenir, mémoire',
 'télépathie',
 'thérapie',
 'transgenre',
 'transmission du savoir',
 'hibernation',
 'instinct, education',
 'nourriture, qui mange qui ?',
 'reproduction animale',
 'trace',
 'zoologie',
 'arbre',
 'champignons',
 'fleur',
 'fruit',
 'légume',
 'catastrophe naturelle',
 'intempérie',
 'météo',
 'neige',
 'nuage',
 'pluie, orage',
 'soleil',
 'vent',
 'automne',
 'cycle des saisons',
 'été',
 'heure',
 'hiver',
 'printemps',
 'semaine',
 'biologie',
 'déchets',
 'eau',
 'écologie, environnement',
 'espace, air',
 'feu',
 'métamorphose (biologie)',
 'naturaliste',
 'nuit (dans la)',
 'pierre, caillou, minéral',
 'planète terre',
 "protection de l'environnement",
 'reproduction végétale',
 'vie sous-marine',
 'vie souterraine',
 'astronomie',
 'aventure aérienne et spatiale',
 'ciel',
 'conquête spatiale',
 'cosmonaute, spationaute',
 'étoile',
 'lune',
 'classification des espèces',
 'imprimerie',
 'savant(e), femme/homme de science, inventeur',
 'technologie',
 'avion',
 'bateau',
 'camion, voiture',
 'code de la route',
 'engins divers',
 'métro',
 'montgolfière',
 'moyen de transport',
 'port',
 'train',
 'archéologie',
 'architecture',
 'clonage humain',
 'informatique, internet',
 'physique, chimie',
 'sismologie',
 'spéléologie',
 'transformation des produits naturels',
 'internet',
 'médias',
 'publicité',
 'réseaux sociaux',
 'télévision',
 'conditions sociales',
 'pauvreté, précarité',
 'travail (conditions de)',
 'sectes',
 'sorcellerie',
 'Bohémien, Gitan',
 'bonbon',
 'chapeau',
 'cuisine',
 'culture, tradition',
 'gourmandise',
 'nourriture',
 'vêtement',
 'comportements sociaux',
 'anniversaire et fête',
 'carnaval',
 'fête foraine',
 'galette des rois',
 'halloween',
 'Noël',
 'Noël (sapin de)',
 'Pâques',
 'père noël',
 "pique-nique, goûter d'enfants",
 'bricolage',
 'camping',
 'jeux de rôles',
 'jeux vidéo',
 'livre, lecture',
 'manège',
 'scoutisme',
 'esclavage',
 'exil',
 'immigration, migration',
 'intégration',
 'réfugié, sans-papier',
 'apartheid',
 'communisme',
 'conscience politique',
 'démocratie',
 'dictature',
 'Khmers rouges',
 'totalitarisme',
 'arche de noé',
 'bible',
 'foi',
 'intégrisme',
 'juive (culture)',
 'religions, croyances',
 'arts martiaux',
 'boxe',
 'équitation',
 'exploit sportif',
 'football',
 'Jeux olympiques',
 'natation',
 'patin à glace',
 'roller',
 'rugby',
 'ski',
 'sport',
 'surf',
 'tennis',
 'vélo',
 'yoga',
 'inceste',
 'interdit',
 'justice',
 'maltraitance',
 'prison',
 'violence',
 'violence sexuelle',
 'voleur',
 'argent (système économique)',
 'argent de poche',
 'baby-sitting',
 'boulanger, pâtissier',
 'casting',
 'chômage',
 'coiffeur, coiffure',
 'détective',
 'docteur, médecin',
 'écrivain, écriture',
 'explorateur',
 'facteur',
 'journalisme',
 'juge',
 'métiers divers',
 'péniche, marinier',
 'petits boulots',
 'pompier',
 'travail',
 'travail des enfants',
 'vétérinaire',
 'colonie de vacances, centre de loisirs',
 'tour du monde',
 'vacances',
 "vacances à l'étranger",
 'vacances à la campagne',
 'vacances à la mer',
 'vacances à la montagne',
 'voyage',
 'argent',
 'citoyenneté',
 'communication',
 'confinement',
 'piscine',
 'pouvoir',
 'transhumanisme',
 'campagne',
 'ferme',
 'pêche',
 'retour à la nature',
 'vie rurale',
 'mer',
 'naufrage',
 'pirate, corsaire',
 'plage, bord de mer',
 'conditions de vie dans les villes',
 'jardin public',
 'magasin',
 'marché',
 'musée',
 'poste',
 'rue',
 'urbanisme',
 'ville (la)',
 'zoo',
 'cabane',
 'jardin',
 'lieux et territoires, campagne',
 'maison, habitation']

In [5]:
ecole_des_loisirs_pretraite = [word.lower() for word in ecole_des_loisirs]

In [11]:
ecole_des_loisirs_pretraite

['âne',
 'animaux des champs et des bois',
 'animaux divers',
 'animaux domestiques',
 'araignée',
 'autruche, émeu',
 'baleine',
 'belette',
 'bison',
 'blaireau',
 'caméléon',
 'canard, oie',
 'castor',
 'cervidés',
 'chameau, dromadaire',
 'chamois',
 'chat',
 'chauve souris',
 'cheval',
 'chien',
 'cochon',
 'coyote, chacal',
 'crabe',
 'crocodile, alligator',
 'dauphin',
 'des animaux et des hommes',
 'dindon',
 'dinosaure (animal)',
 'écureuil',
 'éléphant',
 'escargot',
 'félins divers',
 'girafe',
 'gnou',
 'grenouille, crapaud, têtard',
 'hamster, cobaye',
 'hérisson, porc-épic',
 'hibou',
 'hippocampe',
 'hippopotame',
 'insectes divers',
 'kangourou',
 'koala',
 'lamantin',
 'lapin, lièvre',
 'lézard',
 'lion',
 'loir, lérot',
 'loup',
 'loutre',
 'lynx',
 'mammouth',
 'marmotte',
 'morse',
 'mouton, chèvre, bouc',
 'oiseaux divers',
 'opossum',
 'ornithorynque',
 'orque',
 'ours',
 'panda',
 'papillon',
 'paresseux (animal)',
 'phoque',
 'pieuvre, poulpe',
 'pingouin, manch

In [6]:
len(ecole_des_loisirs_pretraite)

744

In [7]:
ricochet = ["Abandon/Sentiment d'abandon",
 'Abécédaire/Alphabet',
 'Abeille',
 'Aborigène',
 'Absence',
 'Absurde',
 'Abus sexuel',
 'Accident',
 'Activités manuelles',
 "Activités/Cahier d'activités",
 "Adaptation (d'œuvre littéraire)",
 'Adolescence',
 'Adoption',
 'Adultère',
 'Affection/Tendresse',
 'Affirmation de soi',
 'Afghanistan',
 'Afrique',
 'Afrique du Nord',
 'Afrique du Sud',
 'Aide humanitaire',
 'Alaska',
 'Album',
 'Album sans texte',
 'Alcool/Alcoolisme',
 'Algérie',
 'Alimentation/Goût',
 'Allemagne',
 'Alphabétisation',
 'Alpinisme',
 'Amérique du Nord',
 'Amérique latine',
 'Amitié',
 'Amnésie',
 'Amour',
 'Analyse de texte',
 'Ane',
 'Ange',
 'Animaux',
 'Anniversaire/Fête',
 'Anorexie/Boulimie',
 'Antilles/Caraïbes',
 'Antiquité (Gaule)',
 'Antiquité (Grèce)',
 'Antiquité (Rome)',
 'Antisémitisme',
 'Apartheid',
 'Apprentissage',
 'Apprentissage de la vie',
 'Araignée',
 'Arbre',
 'Archéologie',
 'Architecture',
 'Argent/Argent de poche',
 'Argentine',
 'Arménie',
 'Art',
 'Art brut',
 'Art urbain',
 'Asie',
 'Astrologie',
 'Astronomie',
 "Atelier d'écriture",
 'Australie',
 'Autisme',
 'Autobiographie',
 'Automne',
 'Automobile/Voiture',
 'Autriche',
 'Autruche',
 'Aventure',
 'Avion/Aviation',
 'Bain/Toilette',
 'Bande dessinée',
 'Bande dessinée (Histoire)',
 'Bandit/Brigand',
 'Banlieue',
 'Bateau/Voilier',
 'Beauté/Laideur',
 'Bébé',
 'Belgique',
 'Berceuse',
 'Bestiaire',
 'Bêtise/Désobéissance',
 'Bhoutan',
 'Bible',
 'Bibliographie/Sélection de livre',
 'Bibliothèque',
 'Bibliothèque jeunesse',
 'Bicyclette/Vélo',
 'Bien-être animal',
 'Bijou/Diamant',
 'Biographie',
 'Bise/Bisou/Baiser',
 'Bison',
 'Blaireau',
 'Bolivie',
 'Bonheur',
 'Botanique',
 'Bouddhisme',
 'Boxe',
 'Brésil',
 'Bretagne',
 'Bruits/Cris',
 'Bulgarie',
 'Burundi',
 'Caillou/Rocher',
 'Calligraphie',
 'Cambodge',
 'Cameroun',
 'Campagne/Ferme',
 'Camping',
 'Canada',
 'Canard/Oie',
 'Caprice/Entêtement',
 'Carnaval',
 'Castor',
 'Catastrophe naturelle',
 'Catastrophe nucléaire',
 'Cécité',
 'Célébrité',
 'Censure',
 'Cerf-volant',
 'Cerf/Biche',
 'Chameau/Dromadaire',
 'Champignon',
 'Chanson/Chant',
 'Chapeau',
 'Chat',
 'Château',
 'Chaud/Froid',
 'Chauve-souris',
 'Chenille',
 'Cheval/Equitation',
 'Chevalier/Chevalerie',
 'Chien',
 'Chiffre/Nombre/Livre à compter',
 'Chimie',
 'Chine',
 'Chocolat',
 'Chômage',
 'Chouette/Hibou',
 'Ciel',
 'Cinéma/Vidéo',
 'Cinq sens',
 'Cirque',
 'Citation',
 'Citoyenneté/Civisme',
 'Clown',
 'Coccinelle',
 'Cochon/Sanglier',
 'Coiffeur',
 'Colère',
 'Collection/Collectionneur',
 'Collège',
 'Colombie',
 'Colonie de vacances',
 'Colonisation/Colonie',
 'Coloriage',
 'Comédie musicale',
 'Communication',
 'Compétition',
 'Comptine',
 'Concentration',
 'Condamnation à mort',
 'Condition féminine',
 'Condition sociale',
 'Conformisme',
 'Congo',
 "Conquête de l'Ouest américain",
 'Consommation',
 'Conte',
 'Conte (adaptation)',
 'Contraire',
 'Corée',
 'Corps',
 'Correspondance',
 'Corrida',
 'Corse',
 'Couleur',
 'Courage',
 'Crocodile/Alligator',
 'Cuba',
 'Cuisine/Recette',
 'Culpabilité',
 'Curiosité',
 'Cygne',
 'Danemark',
 'Danse/Ballet',
 'Dauphin/Baleine',
 'Débarquement',
 'Découverte',
 'Déguisement',
 'Délinquance',
 'Déménagement',
 'Dent',
 'Dépression',
 'Désert',
 'Désespoir',
 'Design',
 'Dessin',
 'Dessin animé',
 'Détente/Relaxation',
 'Développement durable',
 'Devinette',
 'Diable',
 'Dictature',
 'Différence',
 'Dinosaure',
 'Disparition',
 'Documentaire',
 'Dormir/Coucher',
 'Double',
 'Doudou/Peluche',
 'Dragon',
 'Drogue',
 'Droit',
 "Droits de l'enfant",
 "Droits de l'homme",
 'Dyslexie/Trouble de la lecture',
 'Dystopie',
 'Eau',
 'Ecole',
 'Ecole maternelle',
 'Ecole primaire',
 'Ecologie',
 'Economie/Finance',
 'Ecosse',
 'Ecriture',
 'Ecureuil',
 'Edition',
 'Edition - Petite Edition',
 'Education',
 'Egalité',
 'Egoïsme',
 'Egypte',
 'Egypte ancienne',
 'Elan',
 'Electricité',
 'Eléphant',
 'Emotions',
 'Encyclopédie/Dictionnaire',
 'Energie',
 'Enfance',
 'Enfant des rues',
 'Enfant sauvage',
 'Enigme',
 'Enlèvement/Kidnapping',
 'Ennui',
 'Enquête',
 'Entraide/Solidarité',
 'Environnement',
 'Epreuve/Initiation',
 'Équateur',
 'Escargot',
 'Esclavage',
 'Espace/vaisseaux spatiaux',
 'Espagne',
 'Espionnage/Espion',
 'Espoir',
 'Etats-Unis',
 'Eté',
 'Ethiopie',
 'Ethique/Morale',
 'Etoile',
 'Europe',
 "Europe de l'Est",
 'Excision',
 'Exclusion',
 'Exil',
 'Exploit',
 'Exposition',
 'Extraterrestre',
 'Fable',
 'Famille',
 "Famille d'accueil",
 'Famille recomposée',
 'Fantaisie',
 'Fantastique',
 'Fantôme',
 'Fascisme',
 'Fée',
 'Femme/Homme',
 'Fête foraine',
 'Feu/Incendie',
 'Fille/Garçon',
 'Finlande',
 'Fleur',
 'Football',
 'Forêt',
 'Forme',
 'Fourmi',
 'France',
 'Fruit',
 'Fugue',
 'Généalogie/Arbre généalogique',
 'Générosité',
 'Géographie/Atlas',
 'Girafe',
 'Gitan/Bohémien',
 'Gourmandise',
 'Grand Nord',
 'Grand-Mère',
 'Grand-Père',
 'Grand/Petit',
 'Grande-Bretagne',
 'Grèce',
 'Grenouille/Crapaud',
 'Grève',
 'Grimace',
 'Gros/Maigre',
 'Guatemala',
 'Guerre/Conflit',
 'Guide/Répertoire',
 'Guyane',
 'Habitation/Maison',
 'Halloween',
 "Hamster/Cochon d'Inde",
 'Handicap',
 'Harcèlement',
 'Hérisson',
 'Héros',
 'Heure',
 'Hippopotame',
 'Histoire',
 'Histoire (Moyen-Age)',
 'Histoire (Première Guerre mondiale)',
 'Histoire (Révolution française)',
 'Histoire (Seconde Guerre mondiale)',
 'Histoire (XIX° siècle)',
 'Histoire (XVIII° siècle)',
 'Histoire (XVII° siècle)',
 'Histoire (XVI° siècle)',
 'Histoire (XV° siècle)',
 'Histoire (XXI° siècle)',
 'Histoire (XX° siècle)',
 "Histoire de l'art",
 'Hiver',
 'Holocauste',
 'Homosexualité',
 'Hongrie',
 'Honte/Gêne',
 'Hôpital',
 'Humour',
 'Hygiène/Propreté',
 'Identité',
 'Illettrisme',
 'Illustration',
 'Imagier',
 'Imagination/Fabulation',
 'Immigration/Emigration',
 'Immortalité',
 'Imprimerie',
 'Inca/Aztèque',
 'Inde',
 'Indien',
 'Individualisme',
 'Indonésie',
 'Inégalité/Injustice',
 'Information/Actualité',
 'Insecte',
 'Instrument de musique',
 'Intégration',
 'Interculturalité',
 'Internet',
 'Inuit',
 'Irak',
 'Iran',
 'Irlande',
 'Islande',
 'Israël',
 'Italie',
 'Jalousie',
 'Jamaïque',
 'Japon',
 'Jardin',
 'Jardinage/Potager',
 'Jazz',
 "Jeu d'optique / Illusion d'optique",
 'Jeu de doigts',
 'Jeu de mots',
 'Jeu vidéo',
 'Jeu/Jouet',
 'Jeux Olympiques',
 'Journal intime',
 'Journalisme',
 'Judaïsme',
 'Jumeaux/Jumelles',
 'Justice',
 'Kamishibaï',
 'Kangourou',
 'Koala',
 'Kosovo',
 'La Réunion',
 "Lac/Cours d'eau",
 'Langue des signes',
 'Langue étrangère (apprentissage)',
 'Langue/Langage',
 'Lapin/Lièvre',
 "Lecture de l'image",
 'Légende arthurienne',
 'Légendes/Merveilleux',
 'Légume',
 'Lemming',
 'Léopard',
 'Lever/Réveil',
 'Liban',
 'Liberté',
 'Librairie',
 'Licorne',
 'Lion',
 'Liste',
 'Littérature',
 'Littérature de jeunesse',
 'Lituanie',
 'Livre animé',
 'Livre-jeu',
 'Livre/Lecture',
 'Londres',
 'Loup',
 'Loup-garou',
 'Lumière',
 'Lune',
 'Lutin',
 'Luxembourg',
 'Lycée',
 'Madagascar',
 'Magasin/Commerce',
 'Magie/Magicien',
 'Maladie',
 'Malaisie',
 'Mali',
 'Maltraitance',
 'Mammouth',
 'Manège',
 'Manga',
 'Manipulation',
 'Manipulation génétique/Clonage',
 'Mariage',
 'Mariage forcé',
 'Marionnette/Guignol',
 'Maroc',
 'Masque',
 'Mathématique',
 'Méchanceté/Cruauté',
 'Médecin/Docteur',
 'Médicament',
 'Mélancolie',
 'Mensonge/Vérité',
 'Menstruation/Règles',
 'Mer/Océan',
 'Métamorphose',
 'Météo/Climat',
 'Métier',
 'Métro',
 'Mexique',
 'Mille et une nuits',
 'Mine/Mineur',
 'Misère/Pauvreté',
 'Mode',
 'Monde arabe',
 'Mongolie',
 'Monstre',
 'Montagne',
 'Moquerie',
 'Mort/Deuil',
 'Mouche',
 'Mouette',
 'Moulin',
 'Mouton/Chèvre',
 'Moyen-Orient',
 'Musée',
 'Musique',
 'Mystère',
 'Mythologie',
 'Naissance/maternité',
 'Napoléon',
 'Narrateur/Narration',
 'Nature',
 'Naufrage',
 'Neige',
 'Népal',
 'New York',
 'Noël',
 'Normandie',
 'Norvège',
 'Nourrice/Baby-sitter',
 'Nourriture',
 'Nouvelle-Calédonie',
 'Nouvelle-Zélande',
 'Nouvelles technologies',
 'Nuit',
 'Obésité',
 'Océanie',
 'Odorat',
 'Oeuf',
 'Ogre/Géant',
 'Oiseau',
 'Opéra',
 'Ordinateur/Informatique',
 'Origami',
 'Orphelin/Orphelinat',
 'Ouïe',
 'Ours',
 'Paix/Pacifisme',
 'Pakistan',
 'Palestine',
 'Panda',
 'Papillon',
 'Pâques',
 'Paresse',
 'Parfum',
 'Paris',
 'Parodie',
 'Partage',
 'Paternité',
 'Patience/Impatience',
 'Pays-Bas',
 'Paysage',
 'Pêche/Pêcheur',
 'Pédagogie',
 'Pédophilie',
 'peintre',
 'Peinture',
 'Père Noël',
 'Pérou',
 'Perse',
 'Persévérance',
 'Petite enfance',
 'Peur',
 'Peur du noir/Cauchemar',
 'Philosophie',
 'Phoque/Morse',
 'Photographie',
 'Pingouin/Manchot',
 'Pique-nique',
 'Pirate/Corsaire',
 'Piscine',
 'Planète',
 'Plante',
 'Pluie/Orage',
 'Poésie',
 'Poison',
 'Poisson',
 'Policier',
 'Politesse/Bienséance',
 'Politique',
 'Pollution',
 'Pologne',
 'Polynésie',
 'Pompier',
 'Port',
 'Portrait',
 'Portugal',
 'Poule/Poussin/Coq',
 'Poupée',
 'Pouvoir',
 'Poux',
 'Pratiques artistiques et culturelles',
 'Préhistoire',
 'Préjugés',
 'Prénom/Surnom',
 'Prince/Princesse',
 'Printemps',
 'Prison',
 'Prix littéraire',
 'Promenade',
 'Provence',
 'Proverbe',
 'Psychologie/Psychanalyse',
 'Publicité',
 'Puce',
 'Punition/Fessée',
 'Québec',
 'Questions',
 'Racisme',
 'Racket',
 'Réfugié',
 'Relation Enfant/Adulte',
 'Relation Enfant/Animal',
 'Relation Frère/Soeur',
 'Relation Grand-Mère/Enfant',
 'Relation Grand-Parent/Enfant',
 'Relation Grand-Père/Enfant',
 'Relation Homme/Animal',
 'Relation Mère/Enfant',
 'Relation Mère/Fille',
 'Relation Mère/Fils',
 'Relation Père/Enfant',
 'Relation Père/Fille',
 'Relation Père/Fils',
 'Relation Personne âgée/Enfant',
 'Religion',
 'Renard',
 'Résistance',
 'Respect',
 'Responsabilité',
 'Rêve',
 'Révolte',
 'Richesse',
 'Ritournelle/Répétition',
 'Rivalité/Dispute',
 'Robot',
 'Roi/Reine',
 'Roman',
 'Roman historique',
 'Roman policier',
 'Roumanie',
 'Rue',
 'Rumeur',
 'Ruse',
 'Russie',
 'Rwanda',
 'Sagesse',
 'Saint-Nicolas',
 'Saison/Année/Cycle des saisons',
 'Santé',
 'Sapin',
 'Science',
 'Science-Fiction',
 'Sculpture',
 'Secret',
 'Secte',
 'Sécurité routière',
 'Ségrégation raciale',
 'Sénégal',
 'Séparation/Divorce',
 'Serpent',
 'Sexualité/Education sexuelle',
 'Sida',
 'Silence',
 'Singe',
 'Société',
 'Soleil',
 'Solitude',
 'Sommeil',
 'Son/Eveil sonore',
 'Sorcier/Sorcellerie',
 'Souffrance',
 'Souris/Rat/Mulot',
 'Souvenir/Mémoire',
 'Spéléologie',
 'Spiritualité',
 'Sport',
 'Steampunk',
 'Sucre',
 'Suède',
 'Suicide',
 'Suisse',
 'Super-héros',
 'Superstition',
 'Surdité',
 'Survie',
 'Suspense',
 'Syrie',
 'Tabac/Tabagisme',
 'Taïwan',
 'Taupe',
 'Téléphone',
 'Télévision/Radio',
 'Temps',
 'Terre',
 'Terrorisme',
 'Thaïlande',
 'Théâtre',
 "Théâtre d'ombres",
 'Tibet',
 'Tigre',
 'Timidité',
 'Tolérance',
 'Tortue',
 'Toucher/Matière',
 'Tradition',
 'Traduction',
 'Train/Gare',
 'Transidentité',
 'Transport/Moyens de transport',
 'Travail',
 'Travail des enfants',
 'Trésor',
 'Trisomie',
 'Tristesse/Chagrin',
 'Tsigane',
 'Tunisie',
 'Turquie',
 'Typographie',
 'Tyrannie',
 'Ukraine',
 'Univers/Système solaire',
 'Vacances',
 'Vache',
 'Vampire',
 'Végétarisme/Végétalisme',
 'Vénézuela',
 'Vengeance',
 'Venise',
 'Vent',
 'Vêtement',
 'Vie quotidienne',
 'Vieillesse',
 'Vietnam',
 'Viking',
 'Ville',
 'Viol',
 'Violence',
 'Voisinage',
 'Vol/Voleur',
 'Volcan',
 'Voyage',
 'Vue',
 'Yéti',
 'Yoga',
 'Zèbre',
 'Zombie',
 'Zoo',
 'Zoologie/Naturalisme']

In [8]:
ricochet_pretraite = [word.lower() for word in ricochet]

In [9]:
ricochet_pretraite

["abandon/sentiment d'abandon",
 'abécédaire/alphabet',
 'abeille',
 'aborigène',
 'absence',
 'absurde',
 'abus sexuel',
 'accident',
 'activités manuelles',
 "activités/cahier d'activités",
 "adaptation (d'œuvre littéraire)",
 'adolescence',
 'adoption',
 'adultère',
 'affection/tendresse',
 'affirmation de soi',
 'afghanistan',
 'afrique',
 'afrique du nord',
 'afrique du sud',
 'aide humanitaire',
 'alaska',
 'album',
 'album sans texte',
 'alcool/alcoolisme',
 'algérie',
 'alimentation/goût',
 'allemagne',
 'alphabétisation',
 'alpinisme',
 'amérique du nord',
 'amérique latine',
 'amitié',
 'amnésie',
 'amour',
 'analyse de texte',
 'ane',
 'ange',
 'animaux',
 'anniversaire/fête',
 'anorexie/boulimie',
 'antilles/caraïbes',
 'antiquité (gaule)',
 'antiquité (grèce)',
 'antiquité (rome)',
 'antisémitisme',
 'apartheid',
 'apprentissage',
 'apprentissage de la vie',
 'araignée',
 'arbre',
 'archéologie',
 'architecture',
 'argent/argent de poche',
 'argentine',
 'arménie',
 'art',

In [10]:
len(ricochet)

716

## échantillonnage aléatoire

In [13]:
# éléments dans la v1 de l'ensemble de données (composé de 30 ricochets, 30 écoles des loisirs)
# listes utilisées pour que la v2 de l'ensemble de données ne contiennent pas la v1
ricochet_v1_exclure = ['ours', 'monde arabe', 'naissance/maternité', 'révolte', 'londres', 'papillon', 'souvenir/mémoire', 'eau', 'sénégal', 'dragon', 'hygiène/propreté', "atelier d'écriture", 'alaska', 'poisson', 'astronomie', 'documentaire', 'débarquement', 'guatemala', 'sida', "lecture de l'image", 'pompier', 'nuit', 'univers/système solaire', 'peintre', 'souffrance', 'aide humanitaire', 'maltraitance', 'finlande', 'licorne', 'edition']
ecole_des_loisirs_v1_exclure = ['conte détourné', 'histoire : 18e siècle', 'aventure et mystère', 'théâtre contemporain', 'tchécoslovaquie', 'oman', 'genre', 'recherche de ses origines', 'préhistoire', 'père noël', 'rugby', 'balle, ballon', 'bolivie', 'roller', 'animaux domestiques', 'yémen', 'écrivain, écriture', 'biographie', 'hippocampe', 'arts martiaux', 'semaine', 'livre-jeu', 'histoire sans texte', 'cow-boy', 'transhumanisme', "conte détourné, boucle d'or et les 3 ours", 'relation enfant, adulte', 'conte détourné, le loup et les 7 cabris', 'regard des autres', 'iran']

In [16]:
# Echantillon aléatoire de ricochet_raw en omettant les éléments de omit_list
ricochet_samples = random.sample([word for word in ricochet_pretraite if word not in ricochet_v1_exclure], 20)

print("Echantillon aléatoire de ricochet_raw en omettant les éléments de omit_list")
print(len(ricochet_samples), ricochet_samples)

# Echantillon aléatoire de ecole_des_loisirs_raw en omettant les éléments de omit_list
ecole_des_loisirs_samples = random.sample([word for word in ecole_des_loisirs if word not in ecole_des_loisirs_v1_exclure], 20)

print("Echantillon aléatoire de ecole_des_loisirs_raw en omettant les éléments de omit_list")
print(len(ecole_des_loisirs_samples), ecole_des_loisirs_samples)

Echantillon aléatoire de ricochet_raw en omettant les éléments de omit_list
20 ['sommeil', 'science-fiction', 'provence', 'prince/princesse', 'cuba', 'roi/reine', 'dépression', 'maroc', 'activités manuelles', 'cuisine/recette', 'roman policier', 'tigre', 'histoire (xvii° siècle)', 'poison', 'langue des signes', "théâtre d'ombres", 'antiquité (grèce)', 'grèce', 'mongolie', 'zoologie/naturalisme']
Echantillon aléatoire de ecole_des_loisirs_raw en omettant les éléments de omit_list
20 ['boxe', 'bobo', 'mime', 'Norvège', "classiques du livre d'images", 'crabe', 'conditions sociales', 'les échecs', 'souvenir, mémoire', 'gnou', "protection de l'environnement", 'Brésil', 'totalitarisme', 'images pour jouer', 'dinosaure (animal)', 'argent', 'boulimie', 'Mésopotamie', 'orque', 'comportements sociaux']


## Assignation des évaluateurs
2 évaluateurs par échantillon pour 3 évaluateurs

In [17]:
def assigner_evaluateurs(echantillons):
    evaluateurs = ['Juliette', 'Michel', 'Karim']

    # Create a cycle iterator for the reviewers
    cycles_evaluateurs = itertools.cycle(evaluateurs)

    # Assign reviewers to each sample with a UUID
    assignations = {}
    for sample in echantillons:
        evaluateurs = [next(cycles_evaluateurs) for _ in range(2)] # Assign 2 reviewers per sample
        id_assignation = uuid.uuid4()
        assignations[id_assignation] = {'sample': sample, 'reviewers': evaluateurs}

    return assignations

# Example usage:
assignations_ricochet = assigner_evaluateurs(ricochet_samples)
assignations_ecole_des_loisirs = assigner_evaluateurs(ecole_des_loisirs_samples)


In [18]:
assignations_ricochet

{UUID('224c8294-df7f-49d0-a969-ae2b83acf7e8'): {'sample': 'sommeil',
  'reviewers': ['Juliette', 'Michel']},
 UUID('1a1d8502-9286-4edd-80c8-da575323dadd'): {'sample': 'science-fiction',
  'reviewers': ['Karim', 'Juliette']},
 UUID('25d3bc0f-45d6-46b6-8b79-7157457f355b'): {'sample': 'provence',
  'reviewers': ['Michel', 'Karim']},
 UUID('c0f46e79-5200-48a1-97dc-9d372fff77cf'): {'sample': 'prince/princesse',
  'reviewers': ['Juliette', 'Michel']},
 UUID('8fdf6293-7a63-4fb3-a107-7d6313a56123'): {'sample': 'cuba',
  'reviewers': ['Karim', 'Juliette']},
 UUID('018fc02e-efa8-49a1-a0c7-ce2078850737'): {'sample': 'roi/reine',
  'reviewers': ['Michel', 'Karim']},
 UUID('68b3e3d2-2394-45ad-bd46-e4dd4adf8854'): {'sample': 'dépression',
  'reviewers': ['Juliette', 'Michel']},
 UUID('25a6664a-fca6-4cdb-92cb-6f312e2689f6'): {'sample': 'maroc',
  'reviewers': ['Karim', 'Juliette']},
 UUID('e7446d40-5abf-4aeb-a444-e66d1598322d'): {'sample': 'activités manuelles',
  'reviewers': ['Michel', 'Karim']},
 

In [19]:
assignations_ecole_des_loisirs

{UUID('9ff135bc-d903-4dbe-a0ac-2a7fea81b86a'): {'sample': 'boxe',
  'reviewers': ['Juliette', 'Michel']},
 UUID('0287bd95-9ebd-4872-9b8b-84382feb66e2'): {'sample': 'bobo',
  'reviewers': ['Karim', 'Juliette']},
 UUID('873bf23c-13eb-4297-90a6-6169f4dfd48f'): {'sample': 'mime',
  'reviewers': ['Michel', 'Karim']},
 UUID('d465a4e6-55cf-4507-bfe5-bf262d06df02'): {'sample': 'Norvège',
  'reviewers': ['Juliette', 'Michel']},
 UUID('04c1adf1-c073-479b-9308-fe88615b9780'): {'sample': "classiques du livre d'images",
  'reviewers': ['Karim', 'Juliette']},
 UUID('341f1f54-d665-4932-bb37-50038a1b70e8'): {'sample': 'crabe',
  'reviewers': ['Michel', 'Karim']},
 UUID('72f1d79e-f2d7-4a9a-9e63-62774d362d23'): {'sample': 'conditions sociales',
  'reviewers': ['Juliette', 'Michel']},
 UUID('423161f8-19b6-4bb1-822c-947a6ff72b81'): {'sample': 'les échecs',
  'reviewers': ['Karim', 'Juliette']},
 UUID('b8da26dd-32cf-4193-8d31-b0e8d2b1aed3'): {'sample': 'souvenir, mémoire',
  'reviewers': ['Michel', 'Karim'

## brassage aléatoire des assignations

In [20]:
assignations_brassees = list(assignations_ricochet.items()) + list(assignations_ecole_des_loisirs.items())
random.shuffle(assignations_brassees)

In [21]:
assignations_brassees

[(UUID('68b3e3d2-2394-45ad-bd46-e4dd4adf8854'),
  {'sample': 'dépression', 'reviewers': ['Juliette', 'Michel']}),
 (UUID('9ff135bc-d903-4dbe-a0ac-2a7fea81b86a'),
  {'sample': 'boxe', 'reviewers': ['Juliette', 'Michel']}),
 (UUID('3eaa9a86-91e6-4451-9264-4b60dfeb1676'),
  {'sample': 'totalitarisme', 'reviewers': ['Juliette', 'Michel']}),
 (UUID('c0f46e79-5200-48a1-97dc-9d372fff77cf'),
  {'sample': 'prince/princesse', 'reviewers': ['Juliette', 'Michel']}),
 (UUID('423161f8-19b6-4bb1-822c-947a6ff72b81'),
  {'sample': 'les échecs', 'reviewers': ['Karim', 'Juliette']}),
 (UUID('8fdf6293-7a63-4fb3-a107-7d6313a56123'),
  {'sample': 'cuba', 'reviewers': ['Karim', 'Juliette']}),
 (UUID('25a6664a-fca6-4cdb-92cb-6f312e2689f6'),
  {'sample': 'maroc', 'reviewers': ['Karim', 'Juliette']}),
 (UUID('0287bd95-9ebd-4872-9b8b-84382feb66e2'),
  {'sample': 'bobo', 'reviewers': ['Karim', 'Juliette']}),
 (UUID('8e10d1b2-7452-43f2-8763-4124db7b9d5b'),
  {'sample': 'mongolie', 'reviewers': ['Juliette', 'Michel

## generation du fichier excel pour les evaluateurs

In [25]:
def ecrire_donnees_dans_excel(assignations, evaluateur):
    donnees_a_ecrire = []
    for id_assignations, info_assignation in assignations:
        sample = info_assignation['sample']
        evaluateurs = info_assignation['reviewers']
        if evaluateur in evaluateurs:
            donnees_a_ecrire.append({'UUID': id_assignations,
                        'Echantillon': sample, 
                        'Resultat 1': '', 
                        'Resultat 2': ''})
    
    df = pd.DataFrame(donnees_a_ecrire, columns=['UUID', 'Echantillon', 'Resultat 1', 'Resultat 2'])
    df.to_excel(f"{evaluateur}_assignations_v2.xlsx", index=False)   
    print("donneés ecrites")     


ecrire_donnees_dans_excel(assignations_brassees, 'Juliette')
ecrire_donnees_dans_excel(assignations_brassees, 'Michel')
ecrire_donnees_dans_excel(assignations_brassees, 'Karim')


donneés ecrites
donneés ecrites
donneés ecrites


## excel qui réunit les assignations de tous les evaluateurs 

In [26]:
filename = 'assignations_generales_v2.xlsx' # fonction appelée 2 fois pour peupler le meme fichier
if os.path.exists(filename):
    os.remove(filename)

def peupler_donnees(donnees, assignations, source):
    for id_assignation, info_assignation in assignations.items():
        echantillon = info_assignation['sample']
        evaluateur = info_assignation['reviewers']
        donnees.append({'Uuid': id_assignation,
                     'Source': source, 
                     'Echantillon': echantillon, 
                     'Evaluateur 1': evaluateur[0], 
                     "Evaluateur 2": evaluateur[1], 
                     'Resultat 1': '', 
                     'Resultat 2': ''})
    return donnees
    
def ecrire_donnees_dans_excel_principal():
    # Define the data as a list of dictionaries

    donnees = []   
    donnees = peupler_donnees(donnees, assignations_ricochet, 'ricochet')
    donnees = peupler_donnees(donnees, assignations_ecole_des_loisirs, 'ecoles des loisirs')

    # Write the data to a CSV file
    data_frame = pd.DataFrame(donnees)
    data_frame.to_excel(filename, index=False)
    print("donneés ecrites")     

ecrire_donnees_dans_excel_principal()


donneés ecrites
